from google.colab import drive
drive.mount('/content/drive')


In [2]:
import pandas as pd
pd.set_option('display.max_columns', 1000, 'display.max_colwidth', 1000, 'display.max_rows',1000)

# Read Data

In [13]:
filename='data/df_raw.csv'
# filename='/content/drive/MyDrive/ML3253-Project/df_raw.csv'

df = pd.read_csv(filename)
col=[i.replace(' ','') for i in df.columns]
col=[i.replace('=','') for i in col]
col=[str.lower(i) for i in col]
df.columns=col
df.head(1)

,date,btc-usdopen,clfopen,tslaopen,gspcopen,gsptseopen,ixicopen,btc-usddelta,btc-usddirection,clfdelta,clfdirection,tsladelta,tsladirection,gspcdelta,gspcdirection,gsptsedelta,gsptsedirection,ixicdelta,ixicdirection,tweet_date,content,retweetnum,likenum,userhandle,content1,tweet_day,day,score,negative,neutral,positive,combined,sentiment
0,2022-01-26 09:30:00,38421.363281,86.43,952.429993,4408.430176,20781.300781,13868.869141,0.158282,2.0,-0.103958,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2022-01-25 16:00:26+00:00,"The numbers are in, and the world still loves Adele—even at her worst https://t.co/RIl3yyFA4V https://t.co/mbI3YbCZrI",91,566,business,the number are in and the world still love adeleeven at her worst,Tuesday,Wednesday,"[0.209, 0.51, 0.281, 0.1027]",0.209,0.51,0.281,0.1027,2


In [14]:
# To clean up texts
import re
import nltk
# nltk.download() Download nltk data for first time use (download all packages)
import nltk.data
# from nltk.stem.snowball import SnowballStemmer
# stemmer = SnowballStemmer('english')
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()


# tokenizer = nltk.data.load('nltk:tokenizers/punkt/english.pickle')
nltk.download('wordnet')
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('omw-1.4')

from nltk.corpus import stopwords
stop_words = set(stopwords.words('english'))

def sentence_to_wordlist(sentence, remove_stopwords=False):
    # 0. remove mentions(@), Hashtag(#)
    sentence = re.sub(r'@[^\s]+', '', sentence,flags=re.MULTILINE )
    sentence = re.sub(r'#[^\s]+', '', sentence,flags=re.MULTILINE )
    sentence = re.sub(r'RT[^\s]+', '', sentence,flags=re.MULTILINE )
    # 1. drop http
    p1=re.compile(r'http?:\/\/\S+', flags=re.DOTALL)
    sentence = re.sub(p1, '', sentence)
    # 2. drop https
    p1=re.compile(r'https?:\/\/\S+', flags=re.DOTALL)
    sentence = re.sub(p1, '', sentence)
    # 3. Remove non-letters
    sentence = re.sub(r'[^\w\s]','', sentence)
    # 4. Remove all numbers
    sentence = re.sub(r'[0-9]+', '', sentence)
    # 5. Convert words to lower case and split them
    sentence = sentence.lower().split()
    # Remove Stop Words
    sentence = [word for word in sentence if not word in stop_words]
    # 5. Stemming
    # sentence = [stemmer.stem(w) for w in sentence] 
    # 6. Lemmatizing
    sentence = [lemmatizer.lemmatize(word) for word in sentence]

    #check if returned sentence is blank
    if len(sentence)==0:
      return np.nan
    else:
      # 7. Return a sentence of words
      sentence_r = ''
      for word in sentence:
        sentence_r = sentence_r + ' ' + word
      return(sentence_r)

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


In [15]:
df.content.head(1)

0    The numbers are in, and the world still loves Adele—even at her worst https://t.co/RIl3yyFA4V https://t.co/mbI3YbCZrI
Name: content, dtype: object

In [16]:
df['content2']= df.content.apply(lambda x: sentence_to_wordlist(x))
df['content2'].head(1)

0     number world still love adeleeven worst
Name: content2, dtype: object

In [20]:
df.columns

Index(['date', 'btc-usdopen', 'clfopen', 'tslaopen', 'gspcopen', 'gsptseopen',
       'ixicopen', 'btc-usddelta', 'btc-usddirection', 'clfdelta',
       'clfdirection', 'tsladelta', 'tsladirection', 'gspcdelta',
       'gspcdirection', 'gsptsedelta', 'gsptsedirection', 'ixicdelta',
       'ixicdirection', 'tweet_date', 'content', 'retweetnum', 'likenum',
       'userhandle', 'content1', 'tweet_day', 'day', 'score', 'negative',
       'neutral', 'positive', 'combined', 'sentiment', 'content2'],
      dtype='object')

In [21]:
TARGET = "btc-usddirection"
col = ['content2',TARGET] #,'btc-usddirection','tsladirection','gspcdirection','ixicdirection']
df_1 = df[col]
df_1 = df_1[df_1[TARGET]!= 1] # remove data points when there is no price change
df_1[TARGET].replace(2,1, inplace=True)  # Replace label 2 with 1

df_1 = df_1.dropna()
X = df_1['content2']
y = df_1[col[1]]

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=10086, shuffle=False)

# Text Analysis TFIDF

In [23]:
from sklearn.feature_extraction.text import CountVectorizer
count_vect = CountVectorizer()
X_train_counts = count_vect.fit_transform(X_train)
X_train_counts.shape

(22868, 20887)

In [24]:
from sklearn.feature_extraction.text import TfidfTransformer
tfidf_transformer = TfidfTransformer()
X_train_tfidf = tfidf_transformer.fit_transform(X_train_counts)
X_train_tfidf.shape

(22868, 20887)

# Metrics

In [30]:
class color:
   PURPLE = '\033[95m'
   CYAN = '\033[96m'
   DARKCYAN = '\033[36m'
   BLUE = '\033[94m'
   GREEN = '\033[92m'
   YELLOW = '\033[93m'
   RED = '\033[91m'
   BOLD = '\033[1m'
   UNDERLINE = '\033[4m'
   END = '\033[0m'

def printpp(text,colorc):
      return print(colorc + text + color.END)


In [31]:
from sklearn.metrics import accuracy_score,classification_report,confusion_matrix,precision_score,recall_score, f1_score

def classification_metric(y_test,y_pred):
    # Calculate accuracy
    printpp("accuracy_score %.4f" % accuracy_score(y_test, y_pred),color.GREEN)
    # precision
    printpp("precision_score %.4f"% precision_score(y_test,y_pred),color.GREEN)
    # recall
    printpp("recall_score %.4f"% recall_score(y_test,y_pred),color.GREEN)
    # F-1 score
    printpp("f1_score %.4f"% f1_score(y_test,y_pred),color.GREEN)
    # Confusion Matrix
    print(confusion_matrix(y_test,y_pred))
    # Classification Report
    #print(classification_report(y_test,y_pred))
    return None

def model_evaluation(model,X_train,y_train,X_test,y_test):
    model = model.fit(X_train,y_train)
    # training accuracy
    print('Evaluation for Train Data')
    classification_metric(y_train,model.predict(X_train))

    #testing accuracy
    print('Evaluation for Test Data')
    classification_metric(y_test,model.predict(X_test))
    return None

# Model - Naive Bayes  on Tweets Text

In [26]:
from sklearn.naive_bayes import MultinomialNB
clf = MultinomialNB().fit(X_train_tfidf, y_train)

In [27]:
from sklearn.pipeline import Pipeline
text_clf = Pipeline([('vect', CountVectorizer()),
                      ('tfidf', TfidfTransformer()),
                      ('clf', MultinomialNB()),
])
text_clf = text_clf.fit(X_train, y_train)

In [28]:
import numpy as np
predicted = text_clf.predict(X_test)
np.mean(predicted == y_test)

0.4475928112291749

In [32]:
model_evaluation(text_clf, X_train, y_train, X_test, y_test)

Evaluation for Train Data
accuracy_score 0.7441
precision_score 0.6969
recall_score 0.9591
f1_score 0.8072
[[ 4762  5329]
 [  523 12254]]
Evaluation for Test Data
accuracy_score 0.4476
precision_score 0.4283
recall_score 0.8526
f1_score 0.5702
[[ 619 3728]
 [ 483 2793]]


# SVM-SGD on Tweets Text

In [34]:
from sklearn.linear_model import SGDClassifier
text_clf_svm = Pipeline([('vect', CountVectorizer()),
                          ('tfidf', TfidfTransformer()),
                          ('clf-svm', SGDClassifier(loss='hinge', penalty='l2',
                                               alpha=0.001,random_state=42)),
                        ])

model_evaluation(text_clf_svm, X_train, y_train, X_test, y_test)

Evaluation for Train Data
accuracy_score 0.5587
precision_score 0.5587
recall_score 1.0000
f1_score 0.7169
[[    0 10091]
 [    0 12777]]
Evaluation for Test Data
accuracy_score 0.4298
precision_score 0.4298
recall_score 1.0000
f1_score 0.6012
[[   0 4347]
 [   0 3276]]
